In [1]:
phenotype <- "rd"
clumping <- "Radj2"

In [2]:
suppressMessages(library(tidyverse))
suppressMessages(library(snpStats))
suppressMessages(library(oem))
suppressMessages(library(glmnet))
suppressMessages(library(knockoff))

In [3]:
K <- 50
tmp.dir <- "/scratch/PI/candes/ukbiobank_tmp"
chr.list <- seq(1,22)
n.PCs <- 5

In [4]:
# Load fam file
fam.file <- sprintf("%s/knockoffs/%s_K%d/ukb_gen_chr%d.fam", tmp.dir, clumping, K, 22)
Subjects <- read_delim(fam.file, delim=" ", col_types=cols(),
                       col_names = c("FID", "IID", "X1", "X2", "Sex", "X3"))

# Load list of variants
Variants <- tibble()
for(chr in chr.list) {
    key.file <- sprintf("%s/knockoffs/%s_K%d/ukb_gen_chr%d.key", tmp.dir, clumping, K, chr)
    Variants.chr <- read_delim(key.file, delim=" ", col_types=cols())
    Variants.chr <- Variants.chr %>% mutate(CHR=Chr) %>% select(CHR, Variant, Position, Group, Knockoff)
    colnames(Variants.chr) <- c("CHR", "SNP", "BP", "Group", "Knockoff")
    Variants <- rbind(Variants, Variants.chr)
}

# Load p-values from marginal testing
Results <- tibble()
for(chr in chr.list) {
    pvals.file <- sprintf("%s/association/%s/%s_K%d/ukb_chr%d.assoc",
                         tmp.dir, phenotype, clumping, K, chr)
    Results.chr <- read_table(pvals.file, col_types=cols(), guess_max=5000000)
    Results <- rbind(Results, Results.chr)
}
Results <- Results %>% select(CHR, SNP, BP, P) %>%
    inner_join(Variants, by=c("CHR", "SNP", "BP"))

In [5]:
# Load PLINK lasso results
Lasso.fit <- tibble()
for(chr in chr.list) {
    plink.lasso.file <- sprintf("%s/association/%s/%s_K50/ukb_chr%d.lasso", tmp.dir, phenotype, clumping, chr)
    Lasso.fit.chr <- read.table(plink.lasso.file, sep="", header=T) %>% as.tibble() %>%
        mutate(Importance = abs(EFFECT)) %>% arrange(desc(Importance))
    Lasso.fit <- rbind(Lasso.fit, Lasso.fit.chr)
}

# Combine results with marginal pvalues
Results.lasso <- left_join(Results, Lasso.fit, by=c("CHR", "SNP")) %>%
    select(CHR, SNP, BP, Group, Knockoff, P, Importance) %>%
    arrange(desc(Importance))
head(Results.lasso)

Warning message:
“Column `SNP` joining character vector and factor, coercing into character vector”

CHR,SNP,BP,Group,Knockoff,P,Importance
6,Affx-37072023.A,32612430,272,FALSE,3.659e-94,0.0141280
11,rs61893460.A,76291154,325,FALSE,1.934e-45,0.0135629
15,rs17293632.A,67442596,191,FALSE,9.657e-41,0.0132584
6,rs3104413.B,32582650,271,FALSE,1.698e-69,0.0124420
5,rs1837253.B,110401872,402,FALSE,3.756e-31,0.0116152
9,rs2381416.B,6193455,48,FALSE,4.454e-50,0.0111484


In [11]:
# Compute the knockoff statistics
W.group.stats <- function(importance, knockoff) {
    Z.X  <- sum(importance[which(knockoff==FALSE)], na.rm=T)
    Z.Xk <- sum(importance[which(knockoff==TRUE)], na.rm=T)    
    w = (Z.X-Z.Xk) / sqrt(length(importance))
}
Results.knockoffs <- Results.lasso %>%
    group_by(CHR, Group) %>%
    summarize(W = W.group.stats(Importance,Knockoff), SNP=SNP[1], BP=BP[1], P=min(P)) %>%
    ungroup() %>%
    mutate(Sign.W=factor(sign(W),levels=c(-1,0,1))) %>%
    arrange(desc(abs(W))) %>%
    select(CHR, SNP, BP, Group, W, Sign.W, P)
Results.knockoffs %>% head(10)

CHR,SNP,BP,Group,W,Sign.W,P
10,rs1444782.B,9058671,73,0.002587437,1,6.797e-38
11,rs61893460.A,76291154,325,0.002035109,1,1.934e-45
9,rs2381416.B,6193455,48,0.001890568,1,4.454e-50
2,rs1106639.A,242690675,951,0.001871233,1,3.628e-25
2,rs72823646.A,102954213,410,0.001691947,1,2.638e-43
6,Affx-37072023.A,32612430,272,0.001688619,1,3.659e-94
6,rs3104413.B,32582650,271,0.001512015,1,1.698e-69
5,rs2244012.A,131901225,470,0.001429593,1,1.989e-23
12,rs1059513.B,57489709,243,0.001340117,1,5.397e-22
17,rs4795399.B,38061439,194,0.001308154,1,2.751e-40


In [10]:
# Select groups with the knockoff filter
W.thres <- knockoff.threshold(Results.knockoffs$W, offset=0)
Selected.knockoffs <- Results.knockoffs %>% filter(W >= W.thres)
cat(sprintf("Knockoffs selected %d groups.\n", nrow(Selected.knockoffs)))

Knockoffs selected 157 groups.


In [19]:
# Select groups with marginal pvalues
Selected.marginals <- Results %>% filter(Knockoff==FALSE) %>%
    group_by(CHR, Group) %>%
    summarise(SNP=SNP[which.min(P)], BP=BP[which.min(P)], P=min(P), Group.size=n()) %>%
    select(CHR, SNP, BP, Group, Group.size, P) %>%
    filter(P < 5e-8)
cat(sprintf("Marginal testing selected %d groups.\n", nrow(Selected.marginals)))

Marginal testing selected 130 groups.
